# Scope rules

## This file is optional! 

There are many code-blocks that in some sense work separately from their environment, e.g. `function` definitions, or `for`, `while` and `if` blocks. For example, in

```julia
function add1(x)
    return x+1
end
```

`x` is a local variable, which has no connection to other parts of the code. In every language, it is defined how these local environments work exactly, and the relevant rules are called scope rules. Even though these rules are intended to be intuitive, in my experience:
 - Scoping rules across languages can differ quite a bit.
 - For any language, there are situations where not knowing the scoping rules can drive you crazy.
 - Scoping rules are complex. It's ok to forget them. But if you observe some confusing behavior and any loops or function environments are involved, come back and read again this section. If you don't find an answer, you might want to check the relevant section of the [Documentation](https://docs.julialang.org/en/v1/manual/variables-and-scoping/).

So let's see how these things work in Julia! To establish some vocabulary, consider:

In [1]:
# before running, restart Julia to clean up leftover objects!!!!

a = 1

function f(b)
    c = 2 + a
    function g(d)
        return a*(c + d)
    end
    return g(b)
end

f (generic function with 1 method)

- `a` lives in the global scope.
- `f` introduces a local scope. Its parent scope is the global scope, so it can see variables in the global scope. `b` and `c` live in this local scope and they are invisible from the global scope.
- `g` introduces another local scope inside that of `f`. Inside you can call any variable from the outside, but `d` is invisible from the scope of `f` or from the global scope.

In [2]:
f(5) # this runs

8

variables in the local scopes are invisible from the outside:

In [3]:
b

UndefVarError: UndefVarError: `b` not defined

Let's make `a` an argument of `f`. In this case there are two things called `a`: the global variable defined earlier, and an argument of `f`, living in its local scope.

In [4]:
function f(a,b)
    c = 2 + a
    function g(d)
        return a*(c + d)
    end
    return g(b)
end

f(2,5)

18

Since we get a different result, it is apparent that `f` used `a=2` and not `a=1`. However, `a` in the global scope is still `1`.

In [5]:
a

1

So the `a` in the local vs global scope are entirely different objects! It turns out this behavior is not completely universal. There are two kinds of local scopes, which differ in how they interact with variables defined in an outside scope. 

One more twist: it matters if this outer scope is the global scope of a local one.



In [6]:
a = 1

function test()
    a=2
    return a
end

println("test gives a=$(test()), and outside a=$a holds")

test gives a=2, and outside a=1 holds


In [7]:
function wrap()
    a = 1

    function test()
        a = 2
        return a
    end

    println("test gives a=$(test()), and outside a=$a holds")
    return nothing
end

wrap()
    

test gives a=2, and outside a=2 holds


Hopefully, now you are confused enough, high time to look at the list of the most (all?) important scoping rules in Julia.

1. `function` definitions introduce **hard local** scopes.
2. `for` and `while` loops introduce **soft local** scopes.
3. `if` blocks do **not** introduce a new scope.
4. variables defined in an outer scope are visible in the local scope.
5. new variables **defined** in a local scope are invisible to the outer scopes.
6. if you assign a value to a variable already in use in an outer scope, then we have more complications:
    - if the local scope is **hard** **and** the outer scope in question is the **global** one, then the outer variable is **'shadowed'**. This means that a new local variable is defined by the same name, and for the rest of the local scope the variable from the outer scope becomes invisible.
    - if the local scope is **soft** or the outer scope is also a local one, then the outer variable is **overwritten**. In this case, changes in the local scope affect the variables outside the local scope.

How does this explain the earlier strange phenomena? When `a=2` runs in `test`, it overwrites `a` of the parent scope when the parent scope is the local scope induced by `wrap`, but it does not when the parent is the global scope.

It is apparent from point 6., that the only difference between hard and soft scopes is visible only when the parent scope is the global one. Let's try this.

In [8]:
a = 1

for i in 1:5
    a=i
end

a

5

In [9]:
function wrap()
    a = 1

    for i in 1:5
        a=i
    end

    a
end

wrap()

5

Indeed, this time `a` is overwritten in both cases. This happens so, since `for` introduces a **soft local** scope, in contrast to a `function` definition, which creates a **hard** scope.

##### Example for point 3.

`sign` is created inside the `if` block, but since it does not induce a new scope, `sign` is visible in the global scope.

In [10]:
x = 1

if x>0
    sign = 1
elseif x==0
    sign = 0
else
    sign = -1
end

sign

1